In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import chromedriver_binary
from pymongo import MongoClient 
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
% matplotlib inline

In [8]:
# making connection to mongodb

client = MongoClient()
database_schoolFinds = client['db_schoolFinds']
collection_schoolsFindsPass = database_schoolFinds['collection_schoolFindsPass']
collection_schoolsFindsFail = database_schoolFinds['collection_schoolFindsFail']

In [9]:
driver = webdriver.Chrome()
driver.get("http://www.schoolfinds.com/")
driver.maximize_window()

# finding all the cities links

driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
sleep(3)

selectCityButton = driver.find_element_by_xpath("//div[@class='header-content visible-desktop']//div[@class='span4']//a[2]")
selectCityButton.click()
sleep(2)

allCityLinks = driver.find_elements_by_xpath("//div[@class='modal-body']//ul[@class='last']//a")

closeButton = driver.find_element_by_xpath('//*[@id="popup"]/footer/button')
closeButton.click()
sleep(2)

In [10]:
all_city_links_list = []

for eachCity in allCityLinks:
    all_city_links_list.append(eachCity.get_attribute('href'))
    
all_city_links_list = all_city_links_list[100:150]

In [13]:
all_city_links_list

['http://www.schoolfinds.com/gujarat/surat.html',
 'http://www.schoolfinds.com/gujarat/jamnagar.html',
 'http://www.schoolfinds.com/gujarat/vadodara.html',
 'http://www.schoolfinds.com/gujarat/ahmedabad.html',
 'http://www.schoolfinds.com/west-bengal/darjeeling.html',
 'http://www.schoolfinds.com/west-bengal/parganas.html',
 'http://www.schoolfinds.com/west-bengal/jalpaiguri.html',
 'http://www.schoolfinds.com/west-bengal/kolkata.html',
 'http://www.schoolfinds.com/west-bengal/durgapur.html',
 'http://www.schoolfinds.com/west-bengal/burdwan.html',
 'http://www.schoolfinds.com/west-bengal/siliguri.html',
 'http://www.schoolfinds.com/west-bengal/midnapur.html',
 'http://www.schoolfinds.com/west-bengal/nadia.html',
 'http://www.schoolfinds.com/punjab/jalandhar.html',
 'http://www.schoolfinds.com/punjab/patiala.html',
 'http://www.schoolfinds.com/punjab/kapurthala.html',
 'http://www.schoolfinds.com/punjab/ludhiana.html',
 'http://www.schoolfinds.com/punjab/mohali.html',
 'http://www.schoo

In [14]:
for eachCity in all_city_links_list:
    
    
    print("working for URL ", eachCity)
    driver.get(eachCity)
    sleep(5)
    
    noOfSchoolsTotal = int(re.sub(r"\D", '', str(driver.find_element_by_xpath(r"//div[@class='product-count']").text)))  # 196
    print("Total no of schools are : ", noOfSchoolsTotal)
    
    noOfOlsEstimated = int(noOfSchoolsTotal/10)   # 19 suppose
    print("possible number of ols estimated : ", noOfOlsEstimated)
    
    print("scrolling to page end!!! : for ",noOfOlsEstimated, " times")
    
    for i in range(noOfOlsEstimated):   # cover ols created
        
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        sleep(3)
        
        
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
        
        
        
    if (noOfSchoolsTotal < 10):
        print("no of schools are less than 10")
        driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
    
        
        noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][1]" + "//li[contains(@class,'item')]"))
        print()
        print("No of lis  presentin this single ol :", noOfLisPresent_in_this_ol)
        
        for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"
            
            try:
                schoolAboutDict = {}

                schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                schoolAboutDict['schoolName'] = schoolName

                schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                schoolAboutDict['schoolAddress'] = schoolAddressRefined

                schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                schoolAboutDict['schoolEmail'] = schoolEmail

                schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                schoolAboutDict['schoolContact'] = schoolContact

                schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                schoolAboutDict['schoolURL'] = schoolURL
                
                if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                    print("Already Present in the db")
                    continue

                collection_schoolsFindsPass.insert_one(schoolAboutDict)
                sleep(2)
                
                print("School Details : ", schoolAboutDict)
                print()
                
            except:
                
                print("Error for the head url city : ", eachCity)
                
                schoolDictFail = {}
                schoolDictFail['Error City URL'] = str(eachCity)
                
                if len(str(schoolName) > 0):
                    schoolDictFail['Error School Name'] = str(schoolName)
                
                collection_schoolsFindsFail.insert_one(schoolDictFail)
                
                
        
    else:
        
        if (noOfSchoolsTotal > noOfOlsEstimated*10):   # 196 > 190 
            print("WE NEED AN EXTRAA ITERATION!!!!  AS  :", noOfSchoolsTotal," > " ,noOfOlsEstimated*10)
        
            for i in range(1, noOfOlsEstimated + 2):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19,20

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()
                driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        else:
            print("schools can be exact multiples of 10 ")
            
            for i in range(1, noOfOlsEstimated + 1):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()
                driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        

working for URL  http://www.schoolfinds.com/gujarat/surat.html
Total no of schools are :  25
possible number of ols estimated :  2
scrolling to page end!!! : for  2  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 25  >  20
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 0
-------------------------STARTING FOR OL---------------------------- :  3


No of lis  presentin this ol : 0
working for URL  http://www.schoolfinds.com/gujarat/jamnagar.html
Total no of schools are :  12
possible number of ols estimated :  1
scrolling to page end!!! : for  1  times
WE NEED AN EXTRA

Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
working for URL  http://www.schoolfinds.com/west-bengal/durgapur.html
Total no of schools are :  6
possible number of ols estimated :  0
scrolling to page end!!! : for  0  times
no of schools are less than 10

No of lis  presentin this single ol : 6
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
working for URL  http://www.schoolfinds.com/west-bengal/burdwan.html
Total no of schools are :  22
possible number of ols estimated :  2
scrolling to page end!!! : for  2  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 22  >  20
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the 


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
working for URL  http://www.schoolfinds.com/punjab/kapurthala.html
Total no of schools are :  28
possible number of ols estimated :  2
scrolling to page end!!! : for  2  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 28  >  20
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 10


Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
working for URL  http://www.schoolfinds.com/punjab/moga.html
Total no of schools are :  18
possible number of ols estimated :  1
scrolling to page end!!! : for  1  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 18  >  10
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 8
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present 

Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  3


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db


...

WE NEED AN EXTRAA ITERATION!!!!  AS  : 55  >  50
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 10
School Inserted with success
School Details :  {'schoolName': 'GOVT SR SEC SCHOOL', 'schoolAddress': ': NAMCHI SOUTH SIKKIM -737126', 'schoolEmail': 'Email: pellingsss@gmail.com', 'schoolContact': 'Contact Number: 03595 -234', 'schoolURL': 'http://www.schoolfinds.com/govt-sr-sec-school-17741.html', '_id': ObjectId('5b899f32edf2ee226c3a9633')}

School Inserted with success
School Details :  {'schoolName': 'GOVT SECONDARY SCHOOL', 'schoolAddress': ': NEW NAMCHI PO- NAM

School Inserted with success
School Details :  {'schoolName': 'GOVT RANGCHANG SEC SCHOOL', 'schoolAddress': ': RALONG SOUTH SIKKIM -737126', 'schoolEmail': 'Email: khanisirbong.ss@gmail.com', 'schoolContact': 'Contact Number: +91 -9002126055', 'schoolURL': 'http://www.schoolfinds.com/govt-rangchang-sec-school.html', '_id': ObjectId('5b899f77edf2ee226c3a9649')}

School Inserted with success
School Details :  {'schoolName': 'GOVT GIRLS SR. SEC SCHOOL', 'schoolAddress': ': NAMCHI SOUTH SIKKIM -737126', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 2641979', 'schoolURL': 'http://www.schoolfinds.com/govt-girls-sr-sec-school-22794.html', '_id': ObjectId('5b899f7aedf2ee226c3a964a')}

School Inserted with success
School Details :  {'schoolName': 'GOVERNMENT SECONDARY SCHOOL', 'schoolAddress': ': KABRAY- P.O. NAMTHANG BAZAAR- SOUTH SIKKIM -737132', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 9862013085', 'schoolURL': 'http://www.schoolfinds.com/government-secondary-s

Total no of schools are :  57
possible number of ols estimated :  5
scrolling to page end!!! : for  5  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 57  >  50
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  3


No of lis  presentin this 

School Inserted with success
School Details :  {'schoolName': 'GOVT. SR SEC SCHOOL', 'schoolAddress': ': DENTAM WEST SIKKIM -737112', 'schoolEmail': 'Email: sombarey@gmail.com', 'schoolContact': 'Contact Number: 03595 -254247', 'schoolURL': 'http://www.schoolfinds.com/govt-sr-sec-school-17768.html', '_id': ObjectId('5b89a04cedf2ee226c3a966f')}

School Inserted with success
School Details :  {'schoolName': 'GOVT. SR SEC SCHOOL', 'schoolAddress': ': MANGALBARIA TAKUTHANG VIA- NAYA BAZAR WEST SIKKIM -737121', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number:', 'schoolURL': 'http://www.schoolfinds.com/govt-sr-sec-school-17763.html', '_id': ObjectId('5b89a04fedf2ee226c3a9670')}

School Inserted with success
School Details :  {'schoolName': 'GOVT SECONDARY SCHOOL', 'schoolAddress': ': CHAKUNG VIA- NAYA BAZAR WEST SIKKIM -737127', 'schoolEmail': 'Email: sombarey@gmail.com', 'schoolContact': 'Contact Number: 03595 -254247', 'schoolURL': 'http://www.schoolfinds.com/govt-secondary-schoo

School Inserted with success
School Details :  {'schoolName': 'GOVT SEC SCHOOL', 'schoolAddress': ': KHECHUHERI WEST SIKKIM -737113', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number:', 'schoolURL': 'http://www.schoolfinds.com/govt-sec-school-30834.html', '_id': ObjectId('5b89a089edf2ee226c3a9686')}

School Inserted with success
School Details :  {'schoolName': 'GOVT SEC SCHOOL', 'schoolAddress': ': UTTAREY WEST SIKKIM -737122', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number:', 'schoolURL': 'http://www.schoolfinds.com/govt-sec-school-30833.html', '_id': ObjectId('5b89a08cedf2ee226c3a9687')}

-------------------------STARTING FOR OL---------------------------- :  6


No of lis  presentin this ol : 6
School Inserted with success
School Details :  {'schoolName': 'GOVT SEC SCHOOL', 'schoolAddress': ': LINGCHOM WEST SIKKIM -737121', 'schoolEmail': 'Email: SAJANVS@yahoo.com', 'schoolContact': 'Contact Number:', 'schoolURL': 'http://www.schoolfinds.com/govt-sec-school-3083